In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()
import rasterio
from rasterio.crs import CRS
from matplotlib import pyplot as plt
import cv2
import numpy as np
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from PIL import Image
from rasterio.plot import reshape_as_image

### Data files from Indian Space Data Center

In [ ]:
optical_high_res_01 = r'./landingzone_ref/ch2_ohr_nrp_20240229T0921593215_d_img_d18/browse/raw/20240229/ch2_ohr_nrp_20240229T0921593215_b_brw_d18.png'
img_01_tif = r'./final_input/tmc_roi.tif'
img_01_relief_tif = r'./final_input/tmc_roi_slope.tif'


### Load raster and extract ROI

In [ ]:
img1 = cv2.imread(optical_high_res_01)
img_relief = cv2.imread(img_01_relief_tif)
rasterdat = rasterio.open(img_01_tif, "r+")

scale = 0.23 # meter / pixel
img = rasterdat.read()
img = reshape_as_image(img)
img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

img1_relief = rasterio.open(img_01_relief_tif, "r+")
img1_relief = reshape_as_image(img)
img1_relief = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

print(img_relief.shape)
print(img.shape)
print(img1.shape)
print("shape printed")

height, width = img.shape[:2]
print("Img height: " + str(height) + " Img width: " +  str(width))

ylimit = int(0.33 * height)

# img_roi = img[0:1200, 0:1200]
# img_roi = img[0:1000, 0:1000]
img_roi = img
# img_roi_copy1 = img[0:1200, 0:1200]
# img_roi_copy1 = img[0:1000, 0:1000]
img_roi_copy1 = img
img_roi_copy2 = img.copy()
relief_copy = img_relief.copy()

plt.imshow(img_roi)

In [ ]:
plt.imshow(relief_copy)

In [ ]:
# #placing stop points manually for now from hyperspectral analysis results

# cv2.circle(img_roi, (2170, 3535), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (4180, 2540), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (4730, 1080), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (3620, 1450), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (3605, 1985), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (2640, 2020), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (2570, 2510), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (2950, 2455), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (2140, 3120), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (1480, 2700), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (1250, 3050), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (1120, 1700), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (830, 2470), int(40), (255, 0, 0), 7)
# cv2.circle(img_roi, (620, 3360), int(40), (255, 0, 0), 7)

# plt.imshow(img_roi)

### Mask high slope areas in gradient range

In [ ]:

blur = cv2.GaussianBlur(img_roi, (5,5), 0)
blur_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

# create NumPy arrays from the boundaries
lower = np.array([0,0,0], dtype = "uint8")
# upper = np.array([180,255,60], dtype = "uint8")
upper = np.array([105,160,15], dtype = "uint8")

# find the colors within the specified boundaries and apply
mask = cv2.inRange(blur_hsv, lower, upper)  
mask = 255 - mask
img_roi = cv2.bitwise_and(img_roi, img_roi, mask = mask)

img_roi[mask == 0] = (0, 0, 200)
# img_roi[mask == 0] = (255, 255, 255) # wf

img_relief_gray = cv2.cvtColor(img_relief, cv2.COLOR_BGR2GRAY)

# lower_relief = np.array([50,50,50], dtype = "uint8")
# upper_relief = np.array([200,200,255], dtype = "uint8")

# # hsv_relief[mask == 0] = (255, 0, 0)

mask_relief = cv2.inRange(img_relief_gray, 40, 255)  # slope z factor qgis = 20.000000
# # mask_relief = 255 - mask_relief
# img_relief = cv2.bitwise_and(img_relief, img_relief, mask = mask_relief)
# # img_relief[mask != 0] = (0, 0, 0)
# gray_relief = cv2.cvtColor(img_relief, cv2.COLOR_BGR2GRAY)
blur_relief = cv2.GaussianBlur(mask_relief, (21, 21), 0)
canny_edges_relief = cv2.Canny(blur_relief, 1, 6)
canny_edges_3channel_relief = cv2.cvtColor(canny_edges_relief, cv2.COLOR_GRAY2BGR)
canny_edges_3channel_relief *= np.array((1,1,1),np.uint8)

img_roi = cv2.addWeighted(img_roi, 0.9, canny_edges_3channel_relief, 1, 0)
print(img_relief.shape)
print(np.min(img_relief))
print(np.max(img_relief))
plt.imshow(canny_edges_3channel_relief)

In [ ]:
# plt.imshow(img)

### Ridge detection with canny edge algorithm

In [ ]:
gray1 = cv2.cvtColor(img_roi_copy1, cv2.COLOR_BGR2GRAY)
blur1 = cv2.GaussianBlur(gray1, (45, 45), 0)
canny_edges = cv2.Canny(blur1, 4, 10)
canny_edges_3channel = cv2.cvtColor(canny_edges, cv2.COLOR_GRAY2BGR)
canny_edges_3channel *= np.array((1,1,0),np.uint8) # setting color of canny edges to white explicitly, set value 1,0,0 to convert to red
# canny_edges_3channel *= np.array((1,1,1),np.uint8) # setting color of canny edges to white explicitly, set value 1,0,0 to convert to red # wf
print(canny_edges_3channel.shape)
print(img_roi.shape)
img_roi = cv2.addWeighted(img_roi, 0.7, canny_edges_3channel, 0.5, 0.0, 0.0)

plt.imshow(canny_edges_3channel)

### Multi level crater filtering

In [ ]:
gray = cv2.cvtColor(img_roi, cv2.COLOR_BGR2GRAY)
# gray = img
gray_blurred = cv2.blur(gray, (3, 3)) 
# grayImage = np.array(minima_ridges*255).astype('uint8')

# plt.imshow(grayImage)
# grayImage_blurred = cv2.blur(grayImage, (3,3))
# print(uint_img.shape)
print(gray.shape)

########### primary crater filter #######################
detected_circles = cv2.HoughCircles(canny_edges,
                   cv2.HOUGH_GRADIENT, 1, 20, param1 = 35, 
               param2 = 25.5, minRadius = 0, maxRadius = 45) 
      
########### primary crater filter end #######################

########### secondary (finer) crater filter #######################
detected_circles1 = cv2.HoughCircles(gray_blurred,
                   cv2.HOUGH_GRADIENT, 1, 20, param1 = 35.7, 
               param2 = 25.6, minRadius = 0, maxRadius = 27) 
        
# ########### secondary crater filter end #######################

# ########### sieve (ultra fine) crater filter #######################
# detected_circles2 = cv2.HoughCircles(gray_blurred,
#                    cv2.HOUGH_GRADIENT, 1, 20, param1 = 14, 
#                param2 = 14, minRadius = 0, maxRadius = 10) 
        
# ########### sieve crater filter end #######################
  

### Draw crater filters on ROI

In [ ]:

# if detected_circles2 is not None: 
  
#     detected_circles2 = np.uint16(np.around(detected_circles2)) 
  
#     for pt in detected_circles2[0, :]: 
#         a, b, r = pt[0], pt[1], pt[2] 
  
#         # Draw the circumference of the circle. 
#         # cv2.circle(img_roi, (a, b), int(r/2.5), (0, 255, 0), -1)
#         # cv2.circle(img_roi, (a, b), int(r), (0, 255, 0), 1) 

#         # cv2.circle(img_roi, (a, b), int(r-(r/2)), (0, 0, 0), -1) 
  
#         # Draw a small circle (of radius 1) to show the center. 
#         # cv2.circle(img_roi, (a, b), 1, (0, 0, 255), 3) 

blank = np.zeros((5101, 5090, 3))
if detected_circles1 is not None: 
  
    detected_circles1 = np.uint16(np.around(detected_circles1)) 
  
    for pt in detected_circles1[0, :]: 
        a, b, r = pt[0], pt[1], pt[2] 
  
        # Draw the circumference of the circle. 
        # cv2.circle(img_roi, (a, b), int(r/1.1), (255, 255, 255), -1) # wf
        # cv2.circle(img_roi, (a, b), int(r/1.1), (255, 255, 0), 2) 

        # cv2.circle(img_roi, (a, b), int(r-(r/2)), (0, 0, 0), -1) 
  
        # Draw a small circle (of radius 1) to show the center. 
        # cv2.circle(img_roi, (a, b), 1, (0, 0, 255), 3) 

if detected_circles is not None: 
  
    detected_circles = np.uint16(np.around(detected_circles)) 
  
    for pt in detected_circles[0, :]: 
        a, b, r = pt[0], pt[1], pt[2] 
  
        # Draw the circumference of the circle. 
        # cv2.circle(img_roi, (a, b), int(r/1.5), (255, 255, 255), -1) # wf
        cv2.circle(img_roi, (a, b), int(r/1.5), (0, 255, 0), 2) 
        cv2.circle(blank, (a, b), int(r/1.5), (0, 255, 0), 2)

        # cv2.circle(img_roi, (a, b), int(r-(r/2)), (0, 0, 0), -1) 
  
        # Draw a small circle (of radius 1) to show the center. 
        # cv2.circle(img_roi, (a, b), 1, (0, 0, 255), 3) 

plt.imshow(blank)

In [ ]:
plt.imshow(img_roi)

In [ ]:
#placing stop points manually for now from hyperspectral analysis results

cv2.circle(img_roi, (2170, 3535), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (4180, 2540), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (4730, 1080), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (3620, 1450), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (3605, 1985), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (2640, 2020), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (2570, 2510), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (2950, 2455), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (2140, 3120), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (1480, 2700), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (1250, 3050), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (1120, 1700), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (830, 2470), int(40), (255, 0, 0), 7)
cv2.circle(img_roi, (620, 3360), int(40), (255, 0, 0), 7)

plt.imshow(img_roi)

In [ ]:
import pickle

with open("img_roi.nparr", "wb") as img_roi_file:
    pickle.dump(img_roi, img_roi_file)
filter_layer = cv2.subtract(img_roi, img_roi_copy2)
with open("filter_layer.nparr", "wb") as filterlayer_file:
    pickle.dump(filter_layer, filterlayer_file)
# with open("relief_depth_file.nparr", "wb") as depthfile:
#     pickle.dump(relief_copy, depthfile)
plt.imshow(filter_layer)